In [1]:
from urllib.request import urlopen
from urllib.parse import quote
from lxml import etree
from urllib.error import HTTPError

faulty_urls = list()

import json
import csv
import pickle

dir_ = r"C:\Users\Broccoli\Documents\Bayes Hack"

In [2]:
def parse_search_page(address):
    response = urlopen("http://www.socketsite.com/?s=" + '+'.join(address.split(' ')))
    htmlparser = etree.HTMLParser()
    response = etree.parse(response, htmlparser)

    links = response.xpath('//h3[@class="list-heading-tight"]/a/@href')
    titles = response.xpath('//h3[@class="list-heading-tight"]/a/text()')
    excerpts = response.xpath('//p[@class="excerpt"]/text()')
    
    for i in range(0, len(links)):
        article = {'link':links[i], 'title': titles[i], 'excerpt': excerpts[i]}
        check_address = ''.join([s for s in address if not s.isdigit()]).strip()
        if check_address in article['title'].lower() or check_address in article['excerpt'].lower():
            yield article['link']

In [3]:
def parse_article_page(link):
    response = urlopen(link)
    htmlparser = etree.HTMLParser()
    response = etree.parse(response, htmlparser)
    
    date = response.xpath('//small[@class="staff-subtitle muted"]/text()')
    article = ''.join(response.xpath('//div[@class="article_content"]//text()')).strip()
    comments = response.xpath('//div[@class="comment-content"]/p/text()')

    return {'date': date, 'article': article, 'comments': comments}

In [10]:
output = {}
#start, stop = (1000,1500)

development_pipeline_file = r'\San_Francisco_Development_Pipeline_2014_Quarter_4.csv'
housing_inventory_2014_file = r'\housing_inventory_2014.csv'
housing_inventory_2013_file = r'\housing_inventory_2013.csv'
housing_inventory_2012_file = r'\housing_inventory_2012.csv'
housing_inventory_2011_file = r'\housing_inventory_2011.csv'

with open(dir_ + housing_inventory_2011_file) as csvfile:
    reader = csv.DictReader(csvfile)
    row_range = range(start,stop); row_count = 0
    
    for row in reader:
        row_count += 1
#        if row_count < min(row_range): continue
#        if row_count > max(row_range): break
        if "ST_NAME" in row: address = ' '.join([row["ST_NUM"], row["ST_NAME"], row["ST_TYPE"]]).lower()
        elif "NAMEADDR" in row: address = row['NAMEADDR'].lower()
        elif "STDADDRESS" in row: address = row["STDADDRESS"].lower()
        else: print('ERROR! Unrecognized input'); break
        
        articles = []
        for link in parse_search_page(address):
            articles.append(parse_article_page(link))
        
        output.update({address:articles})

pickle.dump(output, open(dir_ + r"\housing_inventory_2011_comments.pkl", "wb")) #%i_%i.pkl" % (start, stop), "wb"))

ERROR! Unrecognized input
